# Monte Carlo Simulation for FE
## IEOR 4703

### Calculating Gamma for Black-Merton-Scholes via Pathwise Estimator (utilizing scaled Sigmoid)

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
# parameters
s0 = 100
K = 100
sig = 0.35
T = 1/12
r = 0.045
q = 0.015

nN = 500000

### Exact values for $\Gamma$

In [3]:
d1 = (np.log(s0/K) + (r - q + sig**2/2)*T)/(sig*np.sqrt(T))
d2 = d1 - sig*np.sqrt(T)

gammaC = np.exp(-q*T)*norm.pdf(d1)/(s0*sig*np.sqrt(T))
gammaP = gammaC
print(gammaP)

0.03932418976084719


In [4]:
np.random.seed(5145612)
z1 = np.random.randn(nN)
#
tmp1 = s0*np.exp((r-q-sig**2/2)*T)
tmp2 = sig*np.sqrt(T)
s = tmp1*np.exp(tmp2*z1)

In [5]:
def calculate1_d2hds02(b, s, K, s0):
    exp_term = np.exp(-b*(s-K))
    sigmoid_term = (1 + exp_term)**2
    return (((b * exp_term) / sigmoid_term) * (s/s0)**2).sum()

In [6]:
def safe_exp(x, clippingLevel):
    # Define a function for safe exponential calculation
    # Limit the range of x to avoid overflow
    x = np.clip(x, -clippingLevel, clippingLevel)
    return np.exp(x)

In [7]:
# Original code with safe exponential calculation
def calculate2_d2hds02(b, s, K, s0, clippingLevel):
    exponent = -b*(s-K)
    exp_term = safe_exp(exponent, clippingLevel)
    sigmoid_term = (1 + exp_term)**2
    return (((b * exp_term) / sigmoid_term) * (s/s0)**2).sum()

### Pathwise Estimator for $\Gamma$

In [8]:
b = np.array([10.0, 20.0, 50.0, 100, 200])
m = len(b)
#
clippingLevel = np.array([10, 20, 50, 100])
n = len(clippingLevel)

gam_pathwise          = np.zeros((m,1))
gam_pathwise_clipping = np.zeros((m,n))
# ---------------------------------
# Pathwise Estimator
for i in range(m):
    d2hds02 = calculate1_d2hds02(b[i], s, K, s0)
    gam_pathwise[i] = np.exp(-r*T) * d2hds02 / nN
    #
    for j in range(n):
        d2hds02 = calculate2_d2hds02(b[i], s, K, s0, clippingLevel[j])
        gam_pathwise_clipping[i,j] = np.exp(-r*T) * d2hds02 / nN

/tmp/ipython-input-13486649.py:3: RuntimeWarning: overflow encountered in square
  sigmoid_term = (1 + exp_term)**2
/tmp/ipython-input-13486649.py:2: RuntimeWarning: overflow encountered in exp
  exp_term = np.exp(-b*(s-K))
/tmp/ipython-input-13486649.py:4: RuntimeWarning: overflow encountered in multiply
  return (((b * exp_term) / sigmoid_term) * (s/s0)**2).sum()
/tmp/ipython-input-13486649.py:4: RuntimeWarning: invalid value encountered in divide
  return (((b * exp_term) / sigmoid_term) * (s/s0)**2).sum()


### Displaying results

In [9]:
print('==================')
print('\tGamma')
print('==================')
print('Exact: %f' % gammaP)
for i in range(m):
    print('Pathwise estimator (sigmoid w/o clipping):', '%0.2f' % b[i], '   %0.6f' % gam_pathwise[i,0])
    for j in range(n):
        print('Pathwise estimator (sigmoid w/  clipping):', '%0.2f' % b[i], '%0.0f' % clippingLevel[j], '%0.6f' %gam_pathwise_clipping[i,j])
    print('')
print('==========================================================================')

	Gamma
Exact: 0.039324
Pathwise estimator (sigmoid w/o clipping): 10.00    0.039482
Pathwise estimator (sigmoid w/  clipping): 10.00 10 0.039902
Pathwise estimator (sigmoid w/  clipping): 10.00 20 0.039482
Pathwise estimator (sigmoid w/  clipping): 10.00 50 0.039482
Pathwise estimator (sigmoid w/  clipping): 10.00 100 0.039482

Pathwise estimator (sigmoid w/o clipping): 20.00    nan
Pathwise estimator (sigmoid w/  clipping): 20.00 10 0.040578
Pathwise estimator (sigmoid w/  clipping): 20.00 20 0.039699
Pathwise estimator (sigmoid w/  clipping): 20.00 50 0.039699
Pathwise estimator (sigmoid w/  clipping): 20.00 100 0.039699

Pathwise estimator (sigmoid w/o clipping): 50.00    nan
Pathwise estimator (sigmoid w/  clipping): 50.00 10 0.042039
Pathwise estimator (sigmoid w/  clipping): 50.00 20 0.039783
Pathwise estimator (sigmoid w/  clipping): 50.00 50 0.039783
Pathwise estimator (sigmoid w/  clipping): 50.00 100 0.039783

Pathwise estimator (sigmoid w/o clipping): 100.00    nan
Pathwise 